# Grater Expectations Tutorial

![Grater Expectations](https://raw.githubusercontent.com/jschra/grater_expectations/dev/docs/images/grater_expectations_background_small.png)

### Introduction
Welcome to the Grater Expectations tutorial! This notebook will help you run through a full example of using Grater Expectations. To repeat on what is already mentioned in the README, note that you need the following to run all the components of the tutorial:

-  Azure account with [Service Principal access keys](https://learn.microsoft.com/en-us/azure/active-directory/develop/app-objects-and-service-principals)
- [Docker Engine](https://docs.docker.com/engine/): to create new images to run on AWS Lambda and push them to ACR
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli): to login to Azure, create an ACR repository and push docker images to ACR
- Python 3.8: It is recommended to use conda ([Miniconda](https://docs.conda.io/en/latest/miniconda.html)) for easy environment creation and management
- [Terraform](https://www.terraform.io/): to spin up a storage account and containers for GE artifacts and the Data Docs website and an Azure function for testing
- IDE (e.g. VS Code, optional): for easier development (not necessarily for notebooks, but definitely for Python files)

If you have these installed, then you are ready to continue with the tutorial!

<hr>

In order to validate your data, Great Expectations is a package that offers a battery-included set of logic to get up-and-running fast. Fully figuring out how Great Expectations works and applying it to your project, however, can be somewhat involved. This is what Grater Expectations and this tutorial help you with!

Grater Expectations makes a few choices for you and offers scripts, configurations and notebooks to get you started. The choices that were made are:

- Great Expectations output will be stored on Azure Blob storage
- The rendered Data Docs site will be stored on Azure Blob storage (hosted as static website)
- You will write your own data loading logic to read data into memory as a pandas DataFrame
- You will write your own set of expectations to test the quality of this data
- The validation logic will be deployed as Docker container via an Azure function

To set you up for the above, you already entered configurations in `testing_config.yml` for the tutorial, which will be used throughout the code to generate AWS services and access them.

<hr>

To get an idea of how Grater Expectations can help you to develop and deploy logic that you can use to test data, a simplified workflow is shown below. 

![Workflow](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/high_level_workflow_azure.png)

The overall idea is to **configure** data testing by using Great Expectations, so that a preset selection of tests can be run over new data at **runtime**.

In order to **configure** Great Expectations, an example dataset representative of future data to be tested is loaded. Using this dataset, multiple tests (*expectations*) are defined and bundled into a set of tests (*expectation suite*). In order to call this expectation suite at runtime, it is connected to a checkpoint. This checkpoint can then be called at runtime to test new data against, the logic of which is developed aw Python script and built into a Docker image. This image is deployed as an Azure function.

At **runtime**, a POST request containing information on which new data to load and test can then be sent to the deployed Function to invoke it. The Function will then load and validate the new data, using the checkpoint and expectation suite previously developed. The results will then be published on a so-called Data Docs website, which end users can then inspect.

In the rest of this tutorial, each of the steps to set up services, configure Great Expectations and validate new data will be detailed.

### Table of contents
This notebook will guide you through each of the steps to get you testing your data as soon as possible. The steps are:

1. [Setting up a virtual environment](#setting-up-a-virtual-environment)
2. [Provisioning storage](#provisioning-storage)
3. [Imports and configurations](#imports-and-configurations)
4. [Initialization of objects](#initialization-of-objects)
5. [Uploading tutorial data to Blob Storage](#uploading-tutorial-data-to-blob-storage)
6. [Loading data](#loading-data)
7. [Generating a batch request, expectation suite and validator](#generating-a-batch-request-expectation-suite-and-validator)
8. [Creating Expectations](#writing-expectations)
9. [Finalizing expectation suite and creating checkpoint](#finalizing-expectation-suite-and-creating-checkpoint)
10. [Instantiating the Data Docs website](#instantiating-the-data-docs-website)
11. [Developing logic for the Azure Function](#developing-logic-for-the-azure-function)
12. [Creating a Docker image and deployment on Azure ACR](#creating-a-docker-image-and-deployment-on-azure-acr)
13. [Deploying the Azure Function](#deploying-the-azure-function)
14. [Validating new batches of data](#validating-new-batches-of-data)
15. [Wrap up of tutorial and clean-up of Azure services](#wrap-up-of-tutorial-and-clean-up-of-azure-services)
16. [End of tutorial](#end-of-tutorial)

<br>
<hr>


#### Setting up a virtual environment
In order to run the logic contained within this notebook, make sure that it was started up from a virtual environment that contained all required Python dependencies. The easiest way to assure this is to first make a virtual environment for the project and then install `grater_expectations` within it.

To create a new virtual environment, e.g. for python 3.8, and installing the package and its dependencies, run the following:

<br>

**Option 1: Pip**

```bash
# Create a virtual environment
python -m venv env

# Activate the virtual environment
env/Scripts/Activate # Windows
source env/bin/activate # MacOS

# Install into the virtual envirpnment
pip install grater_expectations
```

<br>

**Option 2 - Anaconda**

```bash
# Create a conda environment
conda create --name grater_expectations python=3.8

# Activate the conda environment
conda activate grater_expectations

# Install into the virtual environment
conda install grater_expectations
```

<br>
<hr>

#### Provisioning storage
For testing, Grater Expectations is configured to interact with containers in a storage account on AWS. To be able to run all the code in this notebook the **storage_account** and **data_container_name** as configured in the testing_config.yml must be provisioned, along with the **resource_group_name** that will be provisioned to deploy services in.

To do so, auto-generated Terraform files can be found in the *terraform/storage* directory of this project. To use these configurations to generate storage in that directory, open a (Git bash) terminal (if you didn't already do so), set your Azure programmatic access credentials as environment variables and run the following commands:

<br>

```bash
# Set credentials
export ARM_CLIENT_ID="<enter_client_id_here>"
export ARM_CLIENT_SECRET="<enter_client_secret_here>"
export ARM_SUBSCRIPTION_ID="<enter_subscription_id_here>"
export ARM_TENANT_ID="<enter_tenant_id_here>"

# Go to correct Terraform directory
cd terraform/storage

# Initialize Terraform
terraform init

# Generate deployment plan, enter yes at the prompt if the plan is correct
terraform apply
```
<br>

Terraform will then provide you with a plan, which it will deploy if you enter 'yes' at the prompt as shown below.

![Terraform bucket prompt](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/tutorial_tf_storage_prompt.png)

After entering yes, Terraform will deploy the buckets and will show you the following output when finished.

![Terraform bucket apply](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/tutorial_tf_storage_apply.png)

<br>

You can check if the storage was successfully created by logging in to the portal and following [this link](https://portal.azure.com/#view/HubsExtension/BrowseResource/resourceType/Microsoft.Storage%2FStorageAccounts)

<br>
<hr>

#### Imports and configurations
In the cell below, packages are imported and configurations are loaded from the `project_config.yml`, which was automatically created by `initialize_project.py`. This config file is based on the parameters set in `testing_config.yml` at the root of this repository

Note specifically the import of load_csv_from_container as load_data. For this tutorial, a function was already developed to load csv files from a Blob Storage and turn these into pandas DataFrames. You can inspect the code for this function in `supporting_functions.py` or by calling `help(load_data)`

In [ ]:
# Imports of Python and Grater Expectations packages and logic
import great_expectations as ge
from azure.identity import DefaultAzureCredential
from azure.mgmt.storage import StorageManagementClient 
from azure.storage.blob import BlobServiceClient
from great_expectations.core.batch import RuntimeBatchRequest
from supporting_functions import (TestingConfiguration,
                                  copy_tf_env_vars_for_az,
                                  get_connection_string,
                                  add_connection_string_to_config,
                                  print_ge_site_link,
                                  generate_link_in_notebook,
                                  get_file_keys_from_container)
from supporting_functions import load_csv_from_container as load_data
import json
import os
import requests


# -- Copy environment variables to Azure equivalents for python SDK
copy_tf_env_vars_for_az()

# -- Load parameters from configuration file
test_config = TestingConfiguration("project_config.yml")
test_config.load_config()

In order to access data in the storage account, below credentials are retrieved and an object is initialized. Using this object, a connection string is retrieved for the storage account, so that it can be used in the Great Expectations configuration file to establish connections.

After running the cell below, you can evaluate this by checking out the `great_expectations/great_expectations.yml` file and check if it now contains connection strings throughout the document.

In [ ]:
# -- Obtain Azure credentials, initialize storage client
credentials = DefaultAzureCredential()
storage_client = StorageManagementClient(credentials, os.environ.get("AZURE_SUBSCRIPTION_ID"))

# -- Add connection string of storage account to GE config
connection_string = get_connection_string(storage_client, test_config)
add_connection_string_to_config(connection_string, test_config)

#### Initialization of objects
Next a Great Expectations DataContext is initialized.

By loading the GE configuration file (`great_expectations.yml` found in the great_expectations subdirectory of this project. The DataContext class defaults to this location and file when called), the `DataContext` object stores important parameters for you to interact with GE and automatically interact with S3 buckets for storing output, pulling checkpoints and generating Data Docs sites. For more information on data contexts, check out [the documentation](https://docs.greatexpectations.io/docs/terms/data_context/)

**NOTE**: If generating the DataContext object generates *invalid store configuration* warnings, you probably forgot to provision the storage for the tutorial. Ensure these are properly provisioned before continuing with the rest of the tutorial

In [ ]:
# -- 1. Initialize GE and S3 objects
context = ge.data_context.DataContext()

#### Uploading tutorial data to Blob Storage

To emulate a *normal* setting, you will now upload the tutorial data found in this repository to the container you provisioned using terraform. In order to do so, the PATH_TUTORIAL_DATA constant is set to the local path to the tutorial data. Next, a `BlobServiceClient` object is initialized using the connection string that was generated before, in order to connect to our storage account.

With this client initialized, the data can then be uploaded to the data container as specified in the `testing_config` file.

The data being uploaded contains information about taxi trips in New York City. More information about it can be found [here](https://registry.opendata.aws/nyc-tlc-trip-records-pds/)

After running the code cell below, you can enter the Azure portal and check the container to see if the data now resides on it.

In [ ]:
# -- Start up client for container
blob_service_client = BlobServiceClient.from_connection_string(conn_str=connection_string)

# -- Path constant
PATH_TUTORIAL_DATA = "./data/"

# -- Upload tutorial data
for dataset in os.listdir(PATH_TUTORIAL_DATA):
    # -- Set path to file
    path_file = os.path.abspath(PATH_TUTORIAL_DATA+dataset)
    path_blob = "data/"+dataset

    # -- Create blob_client
    blob_client = blob_service_client.get_blob_client(container=test_config.data_container_name, blob=path_blob)

    with open(path_file, "rb") as data:
        blob_client.upload_blob(data)

#### Loading data

The next step is to load an example dataset that can be used to create expectations for. As previously mentioned, the function `load_csv_from_container` was already developed for this tutorial and was imported as `load_data`. 

Having uploaded the tutorial data to a container in the previous cell, we can now download it and transform it into a pandas DataFrame with the cell below (which is obviously redundant, but implemented for the sake of this tutorial). To know which file to load, we first get a list of objects residing on S3 using `get_file_keys_from_s3` and then pick the oldest one from this list (objects are sorted).

The code of the `load_csv_from_container` function is shown below (note that it can be found in the `supporting_functions.py` file, which is also where this notebook imports it from).

<br>

```python
def load_csv_from_container(
    blob_service_client: BlobServiceClient, container_name: str, path_csv: str
) -> pd.DataFrame:
    """Function that downloads a CSV from a container in an Azure storage account and
    returns it as a pandas DataFrame

    Parameters
    ----------
    blob_service_client : BlobServiceClient
        BlobServiceClient for the storage account to target. Must be authenticated and
        allowed to access and interact with containers
    container_name : str
        Name of the container to get a list of blobs from
    path_csv : str
        Path to the CSV file in the container (can be obtained by calling 
        get_file_keys_from_container)

    Returns
    -------
    pd.DataFrame
        The downloaded CSV file as a pandas DataFrame in memory
    """

    # -- 1. Initiate blob client and download blob
    blob_client = blob_service_client.get_blob_client(
        container=container_name, blob=path_csv
    )
    downloaded_blob = blob_client.download_blob()

    # -- 2. Read incoming stream as pandas DataFrame
    df = pd.read_csv(StringIO(downloaded_blob.content_as_text()))

    return df
```

<br>

Pay special attention to what inputs this function needs and how it knows what file to load, since this will be used in the Azure Function as well. As you can see in the function source code, it will need an initialized `BlobServiceClient`, a container name and a path to a CSV object in order to load data.

Apart from the dataset, Great Expectations needs some additional parameters for operations down the line. These are an identifier for the batch being run and a name for the data asset. These can be the same, as long as they can be used to identify which dataset is being evaluated.

In [ ]:
# -- Get list of blobs in data container, use the first
list_files = get_file_keys_from_container(
    blob_service_client=blob_service_client,
    container_name=test_config.data_container_name,
)
asset_path = list_files[0]
asset_name = asset_path.split("/")[-1]

# -- Load dataset
df_batch = load_data(
    blob_service_client=blob_service_client,
    container_name=test_config.data_container_name,
    path_csv=asset_path,
)
batch_identifier = "tutorial_batch_dataset"


# -- Show top rows of the dataset to get an idea of its contents
df_batch.head(10)

#### Generating a batch request, expectation suite and validator
Now that the data has been loaded, this batch will be passed to a RuntimeBatchRequest below in order to start building an expectation suite. An expectation suite is Great Expectations jargon for a collection of expectations (or tests) that you want to run your data against. 

Great Expectations requires data to be passed as a request when you want to use the context object to generate things such as expectation suites. The RuntimeBatchRequest is used here, because we are loading data (with our own logic) at runtime and want to pass that to subsequent objects.

More information on runtime batch requests can be found [here](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_configure_a_runtimedataconnector/).

Using the RuntimeBatchRequest, two things are done next:
1. Generating an expectation suite: this will serve as a collection of tests you will run for future datasets
2. Generate a validator: this object will use the batch dataset loaded previously to start running expectations and storing these in the suite

As soon as the suite and validator are initiated, you can start writing expectations in the next cells

In [ ]:
# -- 3. Generate batch request at runtime using loaded tile
batch_request = RuntimeBatchRequest(
    datasource_name="runtime_data",
    data_connector_name="runtime_data_connector",
    data_asset_name=asset_name,
    runtime_parameters={"batch_data": df_batch},
    batch_identifiers={"batch_identifier": batch_identifier,},
)

# -- 4. Generate expectation suite, start validator
suite = context.create_expectation_suite(
    test_config.expectations_suite_name,
    overwrite_existing=True,  
)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=test_config.expectations_suite_name,
)

#### Creating Expectations
Using the validator object, expectations can be formulated below. Since Great Expectations comes with many expectations out of the box, [this page](https://greatexpectations.io/expectations) is generally a good place to start browsing through these. 

Predefined expectations can be used by calling them using the validator object and passing the required arguments. For example, to run an expectation on the number of rows in a dataframe, the following snippet can be used:

<br>

```python
# Get number of rows of current batch
row_count = df_batch.shape[0]

# Make expectation where the maximum deviation from the batch number of rows is 1%
max_delta = 0.01
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-max_delta), max_value=row_count * (1+max_delta))
```

<br>

Expectations can be run both on the level of a table, e.g. evaluating the number of rows or columns, and on the level of a column, e.g. evaluating the minimum and maximum within a column. The expected values for such tests are set when generating the expectations suite using the `validator` object in the cells below and will be used for future validations.

Alternatively, expectations can also be set using dynamic evaluation parameters, which is just an expensive set of words for test values that you determine at runtime. This can be useful if you for example want to compare your current dataset with the data of last month and use values in your expectations based on last month's data. An example of how to configure these dynamic evaluation parameters is shown below. More information about them can be found [here](https://docs.greatexpectations.io/docs/reference/evaluation_parameters/)

Apart from existing expectations, you can also develop expectations yourself. If you want to do so, more information can be found about that [here](https://docs.greatexpectations.io/docs/guides/expectations/creating_custom_expectations/overview)

In [ ]:
# -- Table level expectations

# -- Constant parameters
ROW_COUNT_DELTA = .05

# -- 1. Expect future datasets to contain the same columns as current batch DataFrame
expected_columns = df_batch.columns
validator.expect_table_columns_to_match_set(
    column_set=expected_columns, exact_match=True
)

# -- 2. Expect row count to be in between range, based on set delta and number of rows of batch dataset
row_count = df_batch.shape[0]
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-ROW_COUNT_DELTA), 
    max_value=row_count * (1+ROW_COUNT_DELTA),
)

In [ ]:
# -- Column level expectations

# -- 1. Values are never null
for column in expected_columns:
    validator.expect_column_values_to_not_be_null(column)

# -- 2. Date columns are parseable
date_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]
for column in date_columns:
    validator.expect_column_values_to_be_dateutil_parseable(column)

# -- 3. Check dtypes, assuming dtypes of the batch dataset are correct (this is
#       something you might rather want to hard-code for real products)
dict_dtypes = {}
for column, dtype in zip(df_batch.dtypes.index, df_batch.dtypes):
    dict_dtypes[column] = str(dtype)

for column, dtype in dict_dtypes.items():
    validator.expect_column_values_to_be_of_type(column, dtype)

# -- 4. Expect values of specific columns to be between lower- and upper bounds
dict_bounds = {"VendorID":[1,2],
              "payment_type":[1,4]
              }

for column, (lower_bound, upper_bound) in dict_bounds.items():
    # NOTE: for large datasets, this expectation is really slow. Using seperate tests
    # for what the minimum should be and the maximum should be is a lot faster, while
    # achieving the same test-wise
    validator.expect_column_values_to_be_between(column, lower_bound, upper_bound)

In [ ]:
# -- Expectation using dynamic evaluation parameters. Instead of passing values directly 
#    here, a dictionary is passed with the name of the dynamic parameters along with a mock
#    value for the current validation. At runtime, this value will need to be provided 
#    when running the validations

test_column = "passenger_count"
max_passenger_count = df_batch[test_column].max()

validator.expect_column_max_to_be_between(
    column=test_column,
    min_value={
        "$PARAMETER": "min_max_passenger_count",
        "$PARAMETER.min_max_passenger_count": max_passenger_count
        },
    max_value={
        "$PARAMETER": "max_max_passenger_count",
        "$PARAMETER.max_max_passenger_count": max_passenger_count+2
        },
)

#### Finalizing expectation suite and creating checkpoint
After running all the expectations in the cells above, the cell below can be executed to save this set of expectations as a suite and couple it with a checkpoint. 

A checkpoint is an object which can be called by validation logic to run new data batches against, coupling an expectation_suite with parameters to a name. In addition, actions can be coupled to this checkpoint, such as automatically updating the Data Docs website or sending a message on Slack. 

This checkpoint can be used in other scripts (lambda_function.py) by passing a new batch of data (as RuntimeBatchRequest) along with the checkpoint name to the `run_checkpoint` method of an initialized DataContext (which is the same as the `context` object initialized in this notebook). Such a call would look like:

<br>

```python
results = context.run_checkpoint(
    checkpoint_name="CHECKPOINT_NAME",
    validations=[{"batch_request": batch_request}],
    )
```

<br>

When the code below is ran, Great Expectations automatically saves the expectation suite and checkpoint to S3 via the validator and context objects.

In [ ]:
# -- 6. Save suite
validator.save_expectation_suite(discard_failed_expectations=False)

# -- 7.1 Create Simple checkpoint with automatic data docs updates
checkpoint_config = {
    "name": test_config.checkpoint_name,
    "config_version": 3,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": test_config.expectations_suite_name,
    "run_name_template": test_config.run_name_template,
}

# -- 7.2 Add to context object
context.add_checkpoint(**checkpoint_config)

#### Instantiating the Data Docs website
Next, the command below can be ran to build the Data Docs website on a static website hosted on a container on Azure, which provides an interactive user interface in which you can browse through expectation suites and check validation results. More on Data Docs can be found [here](https://docs.greatexpectations.io/docs/reference/data_docs/)

If you use a SimpleCheckpoint, as is the case in this tutorial, the website will automatically be updated each time validations are run. If not, you have to either manually or programmatically update the Data Docs website but calling `context.build_data_docs()`

After initializing the Data Docs website, you should be able to see the expectation suite we just generated and inspect its expectations. In the following steps, we will start running validations, which well then also appear on the website.

In [ ]:
ge_site_output = context.build_data_docs()
print_ge_site_link(ge_site_output)

#### Developing logic for the Azure Function

As previously stated, Grater Expectations implements data testing through deploying testing logic on an Azure Function that can be called over new data. Normally you would have to configure this Function yourself to be able to load data at runtime and run expectations. For this tutorial, the Function code has already been completed with data loading logic and in order for it to run, it expects to receive the path to the data in the container in the parameters of a POST request, so it knows what to load and what to run expectations over at runtime. The parameter that is expected is structured as `{"path_file":<path_to_CSV_in_container>}`


You can find all the code for the Function in `function/grater-expectations/function.py`. To get a better understanding of the function and its steps, it is worthwhile to open this file and walk through the steps and the code. Most of the classes and functions that it uses are stored in `supporting_functions.py`, so it is also worthwhile to have a look there.

In essence, the Function is rather straightforward in its steps:
1. Initialize objects, load configuration files
2. Load data from container to pandas using the `path_file` parameters that was attached to the POST request at runtime
3. Convert the data to a RuntimeBatchRequest
4. Set dynamic evaluation parameters
5. Run validations
6. Evaluate results

As the Function does not require any tweaks, if you understand its contents and functioning, you can proceed to the next step.

<br>
<hr>

#### Creating a Docker image and deployment on Azure ACR
Because there are size constraints when it comes to using Python packages (on AWS) and to ensure that whatever runs locally, also properly runs in the Cloud, Grater Expectations uses Docker containers to deploy logic.

To help you setting this up, `initialize_project.py` automatically generated all the boilerplate code you need to create a Docker image and load it to ACR. Said logic can be found in:
- **Dockerfile**: this file contains the required steps to build a new Docker image to deploy on Azure
- **build_image_store_on_acr.sh**: bash script containing all steps to create a new Docker image using the Dockerfile and load it to ACR, provided you have the Docker Engine and Azure CLI installed and your user credentials (AWS) can be accessed

Since the Azure Function is ready to be Dockerized and deployed at this stage, you can do so by calling this `build_image_store_on_acr.sh` bash script in the terminal from the directory of the tutorial, as shown below. **Make sure that Docker Engine is running before you run the bash script!**

<br>

```bash
# Set AWS credentials in the current terminal
export AWS_ACCESS_KEY_ID=<enter_aws_access_key_here>
export AWS_SECRET_ACCESS_KEY=<enter_aws_secret_access_key_here>

# Go to the directory of the tutorial (if the terminal is not already opened there)
cd tutorial

# Run bash script from tutorial directory
sh build_image_store_on_ecr.sh
```
**NOTE**:
1. For Windows users, the build_image_store_on_acr.sh will not work when called from CMD or Powershell. Instead, use Git Bash (which is automatically installed on your machine when you install Git) to call the bash script. Before doing so, make sure that you export your **Azure access credentials** in the terminal, so it can interact with Azure.

<br>

When `build_image_store_on_acr.sh` is called, the script will build a new Docker image for Python 3.8 using a publicly available base image from Azure, install all dependencies within it based on `requirements.txt` and copy required code- and configuration files onto the image (`supporting_function.py`, `functions/` directory, `project_config.yml` and `great_expectations/great_expectations.yml`). Next, it will create a new repository on Azure ACR (if needed) and upload the Docker image to it. The output in the terminal should look as follows:

![Bash output of deployment](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/bash_output_deployment_Azure.png)

<br>
<hr>

#### Deploying the Azure Function

Now that the Function logic is available as Docker image on ACR, the next step is to deploy an Azure Function that uses this Docker image.

This can be done by using the Terraform configurations that were automatically generated when you ran `initialize_project.py` for the tutorial. In the *terraform/function* subdirectory of the tutorial, you will find the configurations you need to spin up the Function.

Similarly as with the storage, open up a terminal in this directory (or open the directory in the terminal you already have opened) and run the `terraform init` and `terraform plan` commands, typing *yes* when prompted by Terraform. For completeness, see the snippet below.

<br>

```bash
# Set credentials (if not already set in the current terminal)
export ARM_CLIENT_ID="<enter_client_id_here>"
export ARM_CLIENT_SECRET="<enter_client_secret_here>"
export ARM_SUBSCRIPTION_ID="<enter_subscription_id_here>"
export ARM_TENANT_ID="<enter_tenant_id_here>"

# Go to correct Terraform directory
cd terraform/lambda

# Initialize Terraform
terraform init

# Generate deployment plan, enter yes at the prompt if the plan is correct
terraform apply
```
<br>

If successfull, you will see the following outputs.


![Terraform Lambda output](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/tutorial_tf_azure_function.png)

<br>

You can click the link below to check the Azure Function in the Azure console.

In [ ]:
url = f"https://portal.azure.com/#view/HubsExtension/BrowseResource/resourceType/Microsoft.Web%2Fsites/kind/functionapp"
generate_link_in_notebook(url)

#### Validating new batches of data

Now that the validation Function is ready to be used, the next step is to have it validate new batches of taxi trip data.  

To do so, we re-use the list of prefixes to datasets in `list_files` that was previously generated. As you might recall, we used the first dataset in this list as batch dataset to configure expectations for. Now we can use the other datasets to run the expectation suite over using the Lambda function.

To do so, we take the remaining prefixes in the list and generate payloads from them to serve to the Lambda. As previously stated, the Lambda function expects to receive `{"path_file":<path_to_dataset_on_containe>}` in the parameters of the POST request to know which dataset to load.

Therefore, a list of parameters is generated below and stored in `list_params`. This list is subsequently looped over, making a POST request for each element in the list.

In order to make the POST requests, you must retrieve the endpoint of the Azure Function you deployed. This can be done by entering the portal, looking up the Azure Function, selecting the Functions overview under the Functions header and selecting `grater-expectations` (as shown below). When you enter the subsequent window, you can retrieve the endpoint by clicking the `Get Function Url` button. Paste it in the `FUNCTION_ENDPOINT` variable below.

![azure_functions_endpoint](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/azure_functions_function_endpoint.png)

**Note**: *When you have just deployed your Function, it can take some time for it to start up and to show the function in the Functions pane. If there is not function visible yet, wait a bit longer*

After calling the Function, you can check the Data Docs website for the results of running the expectation suite on the other datasets.

In [ ]:
# -- Get list of files to validate (drop all but the first, since we used that before)
list_files_to_validate = list_files[1:]

# -- Set list of params to push in POST requests
list_params = []
for path_file in list_files_to_validate:
    params = {"path_file":path_file}
    list_params.append(params)

# -- Make POST requests for each of the datasets to validate
#    NOTE: The endpoint of the function must be pulled from the Azure portal as soon as the function has been deployed
FUNCTION_ENDPOINT = "THIS_MUST_BE_SET"
list_responses = []
for params in list_params:
    response = requests.post(FUNCTION_ENDPOINT, params=params)
    list_responses.append(response)

# -- Print GE website link for easy access
print_ge_site_link(ge_site_output)

#### Wrap up of tutorial and clean-up of Azure services

If you are able to see the validation runs on the Data Docs website, you have successfully completed the tutorial! You are now equipped and ready to start using Grater Expectations on your own Azure projects. To do so, you can generate a new configuration for a project in `testing_config.yml`, ensure that you fill in parameters for each of the configurations and then call `grater create project --name <project_name>`. A new directory for your project will then be bootstrapped.

Alternatively, if you want to run a project on AWS (or the tutorial for that matter), you can run `grater create config --provider AWS` to create a config file for AWS. Be careful not to overwrite your existing config file though!

Before doing so, you should **clean up the services** that were provisioned for running this tutorial, to prevent unnecessary costs from accruing on your Azure bill.

The easiest way to do so is to open up your terminal again, browse to the *terraform/storage* directory and call `terraform destroy`. Since the resource group for all resources that were provisioned sits in these Terraform configurations, destroying it will ensure all child services are destroyed as well.

<br>

![Terraform Lambda destroy](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/tutorial_tf_storage_destroy.png)

<br>

![Terraform Lambda destroy yes](https://raw.githubusercontent.com/jschra/grater_expectations/main/docs/images/tutorial_tf_storage_destroy_yes.png)

#### End of tutorial
And that concludes this tutorial! If you were running this notebook through a Jupyter server, you can stop it from running by calling the command below.

In [ ]:
os.system("jupyter notebook stop")